In [5]:
import requests
import json

# Replace with your actual OpenWeatherMap API key
API_KEY = '2b73bc66bc71b5e843fd3d853cb0ec42'

# Latitude and longitude for Karachi
lat = 24.8607
lon = 67.0011

# API endpoint
url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_KEY}'

try:
    # Make the request to the API
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        
        # Pretty-print the entire response
        print("Complete API Response:")
        print(json.dumps(data, indent=4))
        
        # Extract specific details (optional)
        air_quality = data.get('list', [])
        if air_quality:
            for index, info in enumerate(air_quality):
                print(f"\nEntry {index + 1}:")
                print(f"  Date and Time: {info['dt']}")
                print(f"  Air Quality Index (AQI): {info['main']['aqi']}")
                print(f"  Components:")
                for component, value in info['components'].items():
                    print(f"    {component.upper()}: {value} µg/m³")
        else:
            print("No air quality data available.")
    else:
        print(f"HTTP Error {response.status_code}")
except Exception as e:
    print(f"An error occurred: {e}")


Complete API Response:
{
    "coord": {
        "lon": 67.0011,
        "lat": 24.8607
    },
    "list": [
        {
            "main": {
                "aqi": 5
            },
            "components": {
                "co": 1428.6,
                "no": 0,
                "no2": 40.44,
                "o3": 65.8,
                "so2": 10.85,
                "pm2_5": 100.92,
                "pm10": 128.74,
                "nh3": 17.73
            },
            "dt": 1737477394
        }
    ]
}

Entry 1:
  Date and Time: 1737477394
  Air Quality Index (AQI): 5
  Components:
    CO: 1428.6 µg/m³
    NO: 0 µg/m³
    NO2: 40.44 µg/m³
    O3: 65.8 µg/m³
    SO2: 10.85 µg/m³
    PM2_5: 100.92 µg/m³
    PM10: 128.74 µg/m³
    NH3: 17.73 µg/m³


In [4]:
# Import necessary libraries
import requests
import json
from datetime import datetime, timedelta
import time

# Configuration
AQI_API_KEY = "2b73bc66bc71b5e843fd3d853cb0ec42"  # AQI API key
WEATHER_API_KEY = "c6831040c0ea40849bc134457252201"  # WeatherAPI key
CITY = "Karachi"

# Geocoding URL to get latitude and longitude
GEO_URL = f"http://api.openweathermap.org/geo/1.0/direct?q={CITY}&limit=1&appid={AQI_API_KEY}"

# Function to fetch coordinates
def fetch_coordinates():
    while True:
        response = requests.get(GEO_URL)
        if response.status_code == 200:
            data = response.json()
            if data:
                return data[0]['lat'], data[0]['lon']
            else:
                raise Exception("City not found.")
        elif response.status_code == 429:  # Too Many Requests
            retry_after = int(response.headers.get("Retry-After", 60))  # Default to 60 seconds
            print(f"Rate limit hit. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
        else:
            raise Exception(f"Failed to fetch coordinates: {response.status_code}")

# Function to fetch historical weather and AQI data
def fetch_historical_data(lat, lon, timestamp):
    # AQI data URL
    AQI_URL = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={timestamp}&end={timestamp+3600}&appid={AQI_API_KEY}"
    # Weather data URL (WeatherAPI)
    WEATHER_URL = f"http://api.weatherapi.com/v1/history.json?key={WEATHER_API_KEY}&q={lat},{lon}&dt={datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')}"
    
    while True:
        # Fetch AQI data
        aqi_response = requests.get(AQI_URL)
        weather_response = requests.get(WEATHER_URL)

        if aqi_response.status_code == 200 and weather_response.status_code == 200:
            aqi_data = aqi_response.json()
            weather_data = weather_response.json()
            return {
                "aqi": aqi_data,
                "weather": weather_data
            }
        elif aqi_response.status_code == 429 or weather_response.status_code == 429:  # Too Many Requests
            retry_after = int(aqi_response.headers.get("Retry-After", 60))  # Default to 60 seconds
            print(f"Rate limit hit for timestamp {timestamp}. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
        else:
            print(f"Failed to fetch data for timestamp {timestamp}: AQI - {aqi_response.status_code}, Weather - {weather_response.status_code}")
            return None

# Fetch coordinates
try:
    latitude, longitude = fetch_coordinates()
except Exception as e:
    print(f"Error fetching coordinates: {e}")
    exit(1)

# Backfill data for the past 2–3 months
end_date = datetime.now()
start_date = end_date - timedelta(days=90)
data = []

# Progress through each hour in the past 90 days
current_date = start_date
while current_date <= end_date:
    timestamp = int(current_date.timestamp())
    historical_data = fetch_historical_data(latitude, longitude, timestamp)
    if historical_data:
        data.append(historical_data)
    current_date += timedelta(hours=1)  # Increment by 1 hour

# Save the backfilled data
with open("historical_aqi_and_weather_data.json", "w") as f:
    json.dump(data, f)

print("Historical AQI and weather data fetched and saved successfully.")


Historical AQI and weather data fetched and saved successfully.
